In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#%cd /content/drive/MyDrive/CODE/VoiceAE/
#!git clone https://github.com/auspicious3000/AutoPST.git

In [13]:
!pip install numpy scipy torch==1.6.0 librosa pysptk soundfile wavenet_vocoder==0.1.1 OpenNMT-py

  Using cached torch-1.6.0-cp37-cp37m-manylinux1_x86_64.whl (748.8 MB)
  Using cached pysptk-0.1.18-cp37-cp37m-linux_x86_64.whl
  Using cached wavenet_vocoder-0.1.1-py3-none-any.whl
  Using cached OpenNMT_py-2.2.0-py3-none-any.whl (216 kB)
  Using cached pyonmttok-1.28.0-cp37-cp37m-manylinux2010_x86_64.whl (15.8 MB)
  Using cached torchtext-0.5.0-py3-none-any.whl (73 kB)
  Using cached waitress-2.0.0-py3-none-any.whl (56 kB)
  Using cached ConfigArgParse-1.5.2-py3-none-any.whl (20 kB)
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torc

In [14]:
%cd /content/drive/MyDrive/CODE/VoiceAE/
!ls AutoPST/assets

/content/drive/MyDrive/CODE/VoiceAE
580000-P.ckpt			  g_03280000	     sea.ckpt
checkpoint_step001000000_ema.pth  mfcc_stats.pkl     spk2emb_82.pkl
cover.png			  p231_p270_001.wav  test_vctk.meta


In [15]:
%cd /content/drive/MyDrive/CODE/VoiceAE/AutoPST/
import os
import pickle
import numpy as np
import torch
import torch.nn.functional as F
from collections import OrderedDict
from onmt.utils.misc import sequence_mask
from model_autopst import Generator_2 as Predictor
from hparams_autopst import hparams

/content/drive/MyDrive/CODE/VoiceAE/AutoPST


In [16]:
device = torch.cuda.current_device()

P = Predictor(hparams).eval().to(device)

checkpoint = torch.load('./assets/580000-P.ckpt', map_location=lambda storage, loc: storage)  
P.load_state_dict(checkpoint['model'], strict=True)
print('Loaded predictor .....................................................')

dict_test = pickle.load(open('./assets/test_vctk.meta', 'rb'))

Loaded predictor .....................................................


### Compute Spectrogram
from new audio file

In [19]:
%cd /content/drive/MyDrive/CODE/VoiceAE/
import os
import pickle
import numpy as np
import librosa
import soundfile as sf
from scipy import signal
from scipy.signal import get_window
from librosa.filters import mel
from numpy.random import RandomState


def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a
    
    
def pySTFT(x, fft_length=1024, hop_length=256):
    
    x = np.pad(x, int(fft_length//2), mode='reflect')
    
    noverlap = fft_length - hop_length
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)
    
    fft_window = get_window('hann', fft_length, fftbins=True)
    result = np.fft.rfft(fft_window * result, n=fft_length).T
    
    return np.abs(result)    
    
    
mel_basis = mel(16000, 1024, fmin=90, fmax=7600, n_mels=80).T
min_level = np.exp(-100 / 20 * np.log(10))
b, a = butter_highpass(30, 16000, order=5)

AUDIO_DIR = "/content/drive/MyDrive/CODE/VoiceAE/audio/"
filename = os.path.join(AUDIO_DIR, "test.wav")

# Read audio file
#x, fs = sf.read(filename) sr=16000
x, fs = librosa.load(filename, duration=4)

# Remove drifting noise
#y = signal.filtfilt(b, a, x)
y = x

# Ddd a little random noise for model roubstness
#wav = y * 0.96 + (prng.rand(y.shape[0])-0.5)*1e-06
wav = y

# Compute spect
D = pySTFT(wav).T

# Convert to mel and normalize
D_mel = np.dot(D, mel_basis)
D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
S = np.clip((D_db + 100) / 100, 0, 1)    

# save spect    
np.save(os.path.join(AUDIO_DIR, filename[:-4]), S.astype(np.float32), allow_pickle=False)

/content/drive/MyDrive/CODE/VoiceAE


don't know how to use this code below yet

it should allow to change speaker

In [17]:
spect_vc = OrderedDict()

uttrs = [('p231', 'p270', '001'),
         ('p270', 'p231', '001'),
         ('p231', 'p245', '003001'),
         ('p245', 'p231', '003001'),
         ('p239', 'p270', '024002'),
         ('p270', 'p239', '024002')]


for uttr in uttrs:
        
    cep_real, spk_emb = dict_test[uttr[0]][uttr[2]]
    cep_real_A = torch.from_numpy(cep_real).unsqueeze(0).to(device)
    len_real_A = torch.tensor(cep_real_A.size(1)).unsqueeze(0).to(device)
    real_mask_A = sequence_mask(len_real_A, cep_real_A.size(1)).float()
    
    _, spk_emb = dict_test[uttr[1]][uttr[2]]
    spk_emb_B = torch.from_numpy(spk_emb).unsqueeze(0).to(device)
    
    with torch.no_grad():
        spect_output, len_spect = P.infer_onmt(cep_real_A.transpose(2,1)[:,:14,:],
                                               real_mask_A,
                                               len_real_A,
                                               spk_emb_B)
    
    uttr_tgt = spect_output[:len_spect[0],0,:].cpu().numpy()
        
    spect_vc[f'{uttr[0]}_{uttr[1]}_{uttr[2]}'] = uttr_tgt

### Spectrogram to Waveform

In [18]:
%cd /content/drive/MyDrive/CODE/VoiceAE/AutoPST/
# Feel free to use other vocoders
# This cell requires some preparation to work, please see the corresponding part in AutoVC
import torch
import soundfile as sf
import pickle
import os
from synthesis import build_model
from synthesis import wavegen

model = build_model().to(device)
checkpoint = torch.load("./assets/checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

#for name, sp in spect_vc.items():
    #print(name, sp.shape)
    #waveform = wavegen(model, c=sp)   
    #librosa.output.write_wav('./assets/'+name+'.wav', waveform, sr=16000)
    #sf.write('./assets/'+name+'.wav', waveform, 16000, 'PCM_24')

# predict
waveform = wavegen(model, c=S)   
sf.write( os.path.join(AUDIO_DIR, 'test_pred.wav'), waveform, 16000, 'PCM_24')

/content/drive/MyDrive/CODE/VoiceAE/AutoPST


<All keys matched successfully>